###Задача:
Использовать полученные знания в части обучения моделей для обучения и подбора параметров в задачах из предыдущих ДЗ на выбор (мфо, задача с тем возьмёт ли человек кредит или нет)
В частности использовать:


GaussianNB
LogisticRegression 
Linear + PolynomialFeatures.
RandomForestClassifier 
При обучении использовать make_pipeline
GridSearchCV для поиска параметров 
Посчитать матрики (f1 + точность + AUC). 

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 80)

In [2]:
# Прочитаем файл данных, полученный после обработки в DZ_07
df = pd.read_csv("./new_credit_train.csv")
df

,Unnamed: 0,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,0,-1.0,-0.68,-1.000000,0.384615,0.258261,-0.222222,-0.791667,0.132031,-0.5,0.047619,-0.009009,-0.142857,-1.0,-1.0
1,1,-1.0,-0.44,-1.000000,0.384615,-0.344083,-0.222222,-0.791667,0.001769,-0.5,0.561905,-0.378378,1.000000,-1.0,-1.0
2,2,1.0,-0.68,-0.333333,0.384615,-0.608184,0.222222,-1.000000,-0.051971,0.5,0.447619,-0.441441,-0.428571,-1.0,-1.0
3,3,1.0,-0.24,-0.333333,0.384615,0.102477,-0.222222,0.250000,0.623152,0.5,-0.847619,0.171171,-0.714286,-1.0,-1.0
4,4,1.0,0.00,-0.333333,-0.692308,0.528326,-0.222222,-0.791667,-0.337911,-0.5,0.790476,-0.009009,1.000000,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133466,133466,-1.0,-0.64,-1.000000,0.384615,0.731802,0.000000,-0.791667,0.194371,-0.5,0.600000,-0.369369,0.714286,-1.0,-1.0
133467,133467,-1.0,-0.76,-0.333333,0.384615,-0.584575,-0.666667,0.250000,0.298665,0.5,0.447619,-0.549550,-0.714286,-1.0,-1.0
133468,133468,-1.0,-0.48,-1.000000,0.384615,-0.308169,-0.222222,-0.333333,-0.132412,-0.5,0.333333,-0.369369,-0.714286,-1.0,-1.0
133469,133469,-1.0,0.40,0.333333,-0.230769,-0.903753,0.000000,-0.791667,0.049950,0.5,0.257143,-0.531532,-0.428571,-1.0,-1.0


In [3]:
#удалим ненужную графу Unnamed: 0	
df = df.drop(['Unnamed: 0'], axis=1)
df

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,-1.0,-0.68,-1.000000,0.384615,0.258261,-0.222222,-0.791667,0.132031,-0.5,0.047619,-0.009009,-0.142857,-1.0,-1.0
1,-1.0,-0.44,-1.000000,0.384615,-0.344083,-0.222222,-0.791667,0.001769,-0.5,0.561905,-0.378378,1.000000,-1.0,-1.0
2,1.0,-0.68,-0.333333,0.384615,-0.608184,0.222222,-1.000000,-0.051971,0.5,0.447619,-0.441441,-0.428571,-1.0,-1.0
3,1.0,-0.24,-0.333333,0.384615,0.102477,-0.222222,0.250000,0.623152,0.5,-0.847619,0.171171,-0.714286,-1.0,-1.0
4,1.0,0.00,-0.333333,-0.692308,0.528326,-0.222222,-0.791667,-0.337911,-0.5,0.790476,-0.009009,1.000000,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133466,-1.0,-0.64,-1.000000,0.384615,0.731802,0.000000,-0.791667,0.194371,-0.5,0.600000,-0.369369,0.714286,-1.0,-1.0
133467,-1.0,-0.76,-0.333333,0.384615,-0.584575,-0.666667,0.250000,0.298665,0.5,0.447619,-0.549550,-0.714286,-1.0,-1.0
133468,-1.0,-0.48,-1.000000,0.384615,-0.308169,-0.222222,-0.333333,-0.132412,-0.5,0.333333,-0.369369,-0.714286,-1.0,-1.0
133469,-1.0,0.40,0.333333,-0.230769,-0.903753,0.000000,-0.791667,0.049950,0.5,0.257143,-0.531532,-0.428571,-1.0,-1.0


In [4]:
# Разделим на обучающую и тестовую выборки
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
X = df.copy()
del X['open_account_flg'] # X - данные (df без open_account_flg)
Y = df['open_account_flg'] # Y - ключи (open_account_flg)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=42)
X_train

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count
59833,1.0,-0.44,-0.333333,0.384615,-0.441055,-0.222222,-0.791667,-0.383863,0.5,0.790476,-0.459459,-0.428571,-1.0
12718,-1.0,-0.80,-0.333333,0.384615,-0.853274,-0.666667,0.875000,0.321830,0.5,0.561905,-0.819820,-0.714286,-1.0
104157,-1.0,-0.12,-0.333333,0.384615,-0.516898,-0.222222,0.250000,0.162617,-0.5,0.295238,-0.549550,-0.428571,-1.0
33729,-1.0,-0.40,1.000000,0.384615,-0.017351,0.000000,0.875000,-0.213028,-0.5,0.447619,-0.423423,-0.142857,-1.0
44374,-1.0,0.16,-1.000000,0.384615,-0.323959,-0.444444,-0.166667,-0.345744,-0.5,0.047619,-0.009009,-0.428571,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,1.0,-0.44,-0.333333,-0.692308,-0.727844,-0.666667,0.041667,-0.700592,-0.5,-0.276190,-0.189189,-0.142857,-1.0
119879,1.0,0.36,-0.333333,0.384615,-0.253690,-0.222222,-0.666667,-0.054760,0.5,0.447619,-0.369369,0.428571,-1.0
103694,1.0,-0.36,1.000000,0.384615,-0.731886,-0.222222,0.875000,0.212940,0.5,-0.219048,-0.549550,-0.714286,-1.0
131932,1.0,-0.56,1.000000,0.384615,-0.170070,0.000000,-0.791667,0.191903,0.5,0.447619,-0.369369,0.428571,-1.0


In [5]:
#GaussianNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
# Выведем точность и матрицу ошибок
print('Точность : ' + str(accuracy_score(y_test, y_pred)))
conf_m = confusion_matrix(y_test, y_pred)
print(conf_m)

Точность : 0.8225245744425798
[[54061  2173]
 [ 9671   831]]


In [6]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
# Классифицируем с помощью RandomForestClassifier
clf = RandomForestClassifier(max_depth=2,
                                    random_state=0)
clf.fit(X_train, y_train) # функция fit обучает модель на тренировочном наборе
y_pred = clf.predict(X_test) # функция predict классифицирует тестовый набор

# Выведем точность и матрицу ошибок

print('Точность : ' + str(accuracy_score(y_test, y_pred)))
conf_m = confusion_matrix(y_test, y_pred)
print(conf_m)

Точность : 0.8426336609925678
[[56234     0]
 [10502     0]]


In [ ]:
#PolynomialFeatures + LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
clf = make_pipeline(PolynomialFeatures(4),
                           LinearRegression())
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('Точность : ' + str(accuracy_score(y_test, y_pred)))
conf_m = confusion_matrix(y_test, y_pred)
print(conf_m)

In [9]:
#GridSearchCV
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
model = SVC() 
model.fit(X_train, y_train) 
   
# print prediction results 
predictions = model.predict(X_test) 
print(classification_report(y_test, predictions)) 

In [8]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Точность : ' + str(accuracy_score(y_test, y_pred)))
conf_m = confusion_matrix(y_test, y_pred)
print(conf_m)

Точность : 0.8426336609925678
[[56234     0]
 [10502     0]]


Вывод:
GridSearchCV и PolynomialFeatures + LinearRegression производили слишком долгие подсчеты, результаты не получены. Среди остальных по точности результаты совпали у LogisticRegression и RandomForestClassifier.